# ML testing: experiment #3 a- unharmonized

This notebook involves testing for the clinical harmonization paper.This notebook shows mixed_dataset (TOP + StrokeMRI) based models without harmonization, applied to various datasets

Data: StrokeMRI, TOP, SABRE, Insight46, EDIS, (HELIUS pending)

Harmonisation: None

Training data: Norment whichh is StrokeMRI and TOP togehter

Testing data: test set from Norment

Futher data applied to: SABRE, Insight46, EDIS, (HELIUS pending)

Validation method: K-fold, double-stratified

Brain-age algorithms: LR, lasso, extra trees, elasticCV net fully tested (but not optimized parameters), additionals partly

Outputs: SubjectID, real age, predicted age of validation and testing sets

### import libraries

In [1]:
import os       # using operating system dependent functionality (folders)
import sys

import glob
from functools import reduce

import pandas as pd # data analysis and manipulation
import numpy as np    # numerical computing (manipulating and performing operations on arrays of data)
import matplotlib.pyplot as plt

# # demo stuff
import ipywidgets as widgets
from ipywidgets import interactive
# import seaborn 

# ml stuff
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
#from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
#from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from sklearn import linear_model
from sklearn import tree
from sklearn import metrics
from sklearn.linear_model import SGDRegressor
from sklearn.metrics import confusion_matrix
from sklearn.metrics import mean_absolute_error
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVR
from sklearn.neural_network import MLPRegressor
from sklearn.linear_model import ElasticNetCV
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.model_selection import cross_val_score

import joblib

sys.path.insert(0, '../') # path to functions
import cvasl.seperated as sep

### import data

In [4]:
# Datasets for this work
EDIS_path = 'our_datasets/EDIS/'
HELIUS_path = 'our_datasets/HELIUS/'
Insight46_path = 'our_datasets/Insight46/'
SABRE_path = 'our_datasets/SABRE/'
MRI_path = 'our_datasets/StrokeMRI/'
TOP_path = 'our_datasets/TOP/'
file_name = 'TrainingDataComplete.csv'

TOP_file = os.path.join(TOP_path, file_name)
MRI_file = os.path.join(MRI_path, file_name)
EDIS_file = os.path.join(EDIS_path, file_name)
#HELIUS_file = os.path.join(HELIUS_path, file_name)
Insight46_file = os.path.join(Insight46_path, file_name)
SABRE_file = os.path.join(SABRE_path, file_name)

EDIS_n = pd.read_csv(EDIS_file)
#HELIUS = pd.read_csv(HELIUS_file)
Insight46_n = pd.read_csv(Insight46_file)
SABRE_n = pd.read_csv(SABRE_file)
TOP_n = pd.read_csv(TOP_file)
MRI_n = pd.read_csv(MRI_file)

In [5]:
# SABRE =SABRE.dropna()
# #SABRE.isna().sum()

In [6]:
Insight46 = Insight46_n.drop(['ID', 'Site'], axis=1)
EDIS = EDIS_n.drop(['ID', 'Site'], axis=1)
SABRE = SABRE_n.drop(['ID', 'Site'], axis=1)
StrokeMRI = MRI_n.drop(['ID', 'Site'], axis=1)

In [7]:
TOP = TOP_n.drop(['ID', 'Site'], axis=1)
TOP.head(2)

,participant_id,Age,Sex,GM_vol,WM_vol,CSF_vol,GM_ICVRatio,GMWM_ICVRatio,WMHvol_WMvol,WMH_count,DeepWM_B_CoV,ACA_B_CoV,MCA_B_CoV,PCA_B_CoV,TotalGM_B_CoV,DeepWM_B_CBF,ACA_B_CBF,MCA_B_CBF,PCA_B_CBF,TotalGM_B_CBF
0,sub-0668_1,50.40,1,0.64252,0.48470,0.26711,0.46081,0.80843,0.004506,18,0.26694,0.17747,0.20996,0.24951,0.24191,24.9902,90.5294,80.9975,57.9326,72.1759
1,sub-0532_1,37.02,1,0.71890,0.59546,0.31236,0.44193,0.80798,0.002501,24,0.25342,0.17620,0.23222,0.21187,0.23518,24.2061,101.2955,90.3336,67.8547,80.1958


In [8]:
TOP.columns = TOP.columns.str.lower() 
StrokeMRI.columns = StrokeMRI.columns.str.lower() 
Insight46.columns = Insight46.columns.str.lower() 
EDIS.columns = EDIS.columns.str.lower() 
SABRE.columns= SABRE.columns.str.lower() 

In [9]:
# check for any duplicated patients between stroke and mri
strokers = set(StrokeMRI.participant_id)
topers = set(TOP.participant_id)
z = strokers.intersection(topers)
print(z) 

set()


In [10]:
# make mixed StrokeMRI and TOP dataset
mixed_data = pd.concat([TOP, StrokeMRI], sort=False)

In [11]:
#give a checkbox for out put folder
loged_feat = widgets.ToggleButton(
    value=False,
    description='Click me if features logged',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Description',
    icon='check' # (FontAwesome names without the `fa-` prefix)
)
loged_feat

ToggleButton(value=False, description='Click me if features logged', icon='check', tooltip='Description')

In [12]:
loged_feat.value

False

In [13]:
if loged_feat.value == False:
    output_folder = '3a_no_log_outputs'
else:
    output_folder = '3a_loged_outputs'

os.makedirs(output_folder, exist_ok=True)

## Build ML models

In [14]:
ml_matrix = mixed_data.drop('participant_id', axis=1)
X = ml_matrix.drop('age', axis =1)
X = X.values
y = ml_matrix['age'].values
y=y.astype('float')

In [52]:
linr_k_frame, linr_y_frame, models = sep.stratified_cat_and_cont_categories_shuffle_split('linear regression', 'unharm_mix_linr', LinearRegression(), ml_matrix, X[:,1:], y)

/scratch/radv/mdijsselhof/cvasl/extended_harm_paper/../cvasl/seperated.py:1055: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  mod_results = pd.concat([mod_results, mod_results_current_fold])
/scratch/radv/mdijsselhof/cvasl/extended_harm_paper/../cvasl/seperated.py:1055: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  mod_results = pd.concat([mod_results, mod_results_current_fold])
/scratch/radv/mdijsselhof/cvasl/extended_harm_paper/../cvasl/seperated.py:1055: FutureWarning: The behavior of DataFra

In [53]:
linr_k_frame

,algorithm,fold,file_name,mae,r2,explained_variance
0,linear regression-0,0,unharm_mix_linr.0,5.649041,0.825896,0.827901
0,linear regression-1,1,unharm_mix_linr.1,5.284237,0.843416,0.843443
0,linear regression-2,2,unharm_mix_linr.2,5.650020,0.825896,0.826470
0,linear regression-3,3,unharm_mix_linr.3,5.393996,0.832842,0.833314
0,linear regression-4,4,unharm_mix_linr.4,5.833636,0.808381,0.808381


In [54]:
linr_k_frame.to_csv(output_folder + '/unharmonized_linr_k_frame.csv')

In [55]:
avg_linr = sep.avg_k_folds(linr_k_frame)
avg_linr

,algorithm,file_name,mae,r2,explained_variance
0,0 linear regression-0 0 linear regressio...,0 unharm_mix_linr.0 0 unharm_mix_linr.1 ...,5.562186,0.827286,0.827902


In [56]:
linr_y_frame

,y_test,y_pred
0,27.550000,35.290096
1,52.495890,51.887689
2,31.372603,46.812958
3,50.345205,56.494202
4,77.593735,61.436322
...,...,...
272,69.424658,82.448777
273,48.730000,45.227547
274,60.183060,55.062832
275,40.355191,40.057050


In [57]:
linr_y_frame.to_csv(output_folder + '/unharmonized_linr_y_frame.csv')

In [58]:
linr = models[0]
linr[0]

LinearRegression()

In [59]:
# check if model folder exists and if not , then create
model_folder = '../result_models/'
if not os.path.exists(model_folder):
    os.makedirs(model_folder)

In [60]:
## optional save models
#joblib.dump(linr[0], ('../result_models/'+ 'unharm_mix_linr0.sav'))
#joblib.dump(linr[1], ('../result_models/'+ 'unharm_mix_linr1.sav'))
#joblib.dump(linr[2], ('../result_models/'+ 'unharm_mix_linr2.sav'))
#joblib.dump(linr[3], ('../result_models/'+ 'unharm_mix_linr3.sav'))
#joblib.dump(linr[4], ('../result_models/'+ 'unharm_mix_linr4.sav'))

In [61]:
llreg_k_frame, llreg_y_frame, models = sep.stratified_cat_and_cont_categories_shuffle_split('lasso regression', 'unharm_mix_llreg',  linear_model.LassoLars(alpha=0.01), ml_matrix, X[:,1:], y)
llreg_k_frame

/scratch/radv/mdijsselhof/cvasl/extended_harm_paper/../cvasl/seperated.py:1055: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  mod_results = pd.concat([mod_results, mod_results_current_fold])
/scratch/radv/mdijsselhof/cvasl/extended_harm_paper/../cvasl/seperated.py:1055: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  mod_results = pd.concat([mod_results, mod_results_current_fold])
/scratch/radv/mdijsselhof/cvasl/extended_harm_paper/../cvasl/seperated.py:1055: FutureWarning: The behavior of DataFra

,algorithm,fold,file_name,mae,r2,explained_variance
0,lasso regression-0,0,unharm_mix_llreg.0,5.590775,0.832415,0.834040
0,lasso regression-1,1,unharm_mix_llreg.1,5.377131,0.837053,0.837125
0,lasso regression-2,2,unharm_mix_llreg.2,5.687867,0.822020,0.823012
0,lasso regression-3,3,unharm_mix_llreg.3,5.337910,0.838348,0.838861
0,lasso regression-4,4,unharm_mix_llreg.4,5.811444,0.808714,0.808749


In [62]:
llreg_k_frame.to_csv(output_folder + '/unharmonized_llreg_k_frame.csv')

In [63]:
avg_llreg = sep.avg_k_folds(llreg_k_frame)
avg_llreg

,algorithm,file_name,mae,r2,explained_variance
0,0 lasso regression-0 0 lasso regression-...,0 unharm_mix_llreg.0 0 unharm_mix_llreg....,5.561026,0.82771,0.828357


In [64]:
llreg_y_frame

,y_test,y_pred
0,27.550000,36.720063
1,52.495890,51.706890
2,31.372603,48.064066
3,50.345205,56.837455
4,77.593735,63.227617
...,...,...
272,69.424658,83.300455
273,48.730000,45.853447
274,60.183060,54.691036
275,40.355191,41.487861


In [65]:
llreg_y_frame.to_csv(output_folder + '/unharmonized_llreg_y_frame.csv')

In [66]:
llreg = models[0]
llreg[0]

LassoLars(alpha=0.01)

In [67]:
## optional save models
#joblib.dump(llreg[0], ('../result_models/'+ 'unharm_mix_llreg0.sav'))
#joblib.dump(llreg[1], ('../result_models/'+ 'unharm_mix_llreg1.sav'))
#joblib.dump(llreg[2], ('../result_models/'+ 'unharm_mix_llreg2.sav'))
#joblib.dump(llreg[3], ('../result_models/'+ 'unharm_mix_llreg3.sav'))
#joblib.dump(llreg[4], ('../result_models/'+ 'unharm_mix_llreg4.sav'))

In [68]:
dtree_k_frame, dtree_y_frame, models = sep.stratified_cat_and_cont_categories_shuffle_split('decision tree', 'unharm_mix_dtree',  tree.DecisionTreeRegressor(), ml_matrix, X[:,1:], y)
dtree_k_frame

/scratch/radv/mdijsselhof/cvasl/extended_harm_paper/../cvasl/seperated.py:1055: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  mod_results = pd.concat([mod_results, mod_results_current_fold])
/scratch/radv/mdijsselhof/cvasl/extended_harm_paper/../cvasl/seperated.py:1055: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  mod_results = pd.concat([mod_results, mod_results_current_fold])
/scratch/radv/mdijsselhof/cvasl/extended_harm_paper/../cvasl/seperated.py:1055: FutureWarning: The behavior of DataFra

,algorithm,fold,file_name,mae,r2,explained_variance
0,decision tree-0,0,unharm_mix_dtree.0,7.264198,0.697288,0.698323
0,decision tree-1,1,unharm_mix_dtree.1,7.335339,0.700662,0.706624
0,decision tree-2,2,unharm_mix_dtree.2,7.194659,0.676318,0.678715
0,decision tree-3,3,unharm_mix_dtree.3,7.084411,0.680746,0.680865
0,decision tree-4,4,unharm_mix_dtree.4,7.531598,0.662953,0.663468


In [69]:
dtree_k_frame.to_csv(output_folder + '/unharmonized_dtree_k_frame.csv')

In [70]:
avg_dtree = sep.avg_k_folds(dtree_k_frame)
avg_dtree

,algorithm,file_name,mae,r2,explained_variance
0,0 decision tree-0 0 decision tree-1 0 ...,0 unharm_mix_dtree.0 0 unharm_mix_dtree....,7.282041,0.683594,0.685599


In [71]:
dtree_y_frame

,y_test,y_pred
0,27.550000,35.580000
1,52.495890,52.732764
2,31.372603,34.130000
3,50.345205,72.770559
4,77.593735,70.912329
...,...,...
272,69.424658,71.287671
273,48.730000,45.200000
274,60.183060,53.136986
275,40.355191,33.710000


In [72]:
dtree_y_frame.to_csv(output_folder + '/unharmonized_dtree_y_frame.csv')

In [73]:
dtree = models[0]
dtree[0]

DecisionTreeRegressor()

In [74]:
regr_k_frame, regr_y_frame, models = sep.stratified_cat_and_cont_categories_shuffle_split('MLP regression', 'unharm_mix_regr',   MLPRegressor(random_state=1, max_iter=700), ml_matrix, X[:,1:], y)
regr_k_frame

/scratch/radv/mdijsselhof/cvasl/extended_harm_paper/../cvasl/seperated.py:1055: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  mod_results = pd.concat([mod_results, mod_results_current_fold])
/scratch/radv/mdijsselhof/cvasl/extended_harm_paper/../cvasl/seperated.py:1055: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  mod_results = pd.concat([mod_results, mod_results_current_fold])
/scratch/radv/mdijsselhof/cvasl/extended_harm_paper/../cvasl/seperated.py:1055: FutureWarning: The behavior of DataFra

,algorithm,fold,file_name,mae,r2,explained_variance
0,MLP regression-0,0,unharm_mix_regr.0,6.712336,0.752517,0.754735
0,MLP regression-1,1,unharm_mix_regr.1,7.981149,0.668760,0.705563
0,MLP regression-2,2,unharm_mix_regr.2,6.902340,0.735490,0.745972
0,MLP regression-3,3,unharm_mix_regr.3,7.622585,0.681550,0.693186
0,MLP regression-4,4,unharm_mix_regr.4,8.020382,0.624955,0.652873


In [75]:
regr_k_frame.to_csv(output_folder + '/unharmonized_regr_k_frame.csv')

In [76]:
avg_regr = sep.avg_k_folds(regr_k_frame)
avg_regr

,algorithm,file_name,mae,r2,explained_variance
0,0 MLP regression-0 0 MLP regression-1 0 ...,0 unharm_mix_regr.0 0 unharm_mix_regr.1 ...,7.447758,0.692654,0.710466


In [77]:
regr_y_frame

,y_test,y_pred
0,27.550000,34.662431
1,52.495890,46.299539
2,31.372603,40.542020
3,50.345205,51.567832
4,77.593735,66.059161
...,...,...
272,69.424658,74.269958
273,48.730000,48.336355
274,60.183060,43.780042
275,40.355191,46.312149


In [78]:
regr_y_frame.to_csv(output_folder + '/unharmonized_regr_y_frame.csv')

In [79]:
regr = models[0]
regr[0]

MLPRegressor(max_iter=700, random_state=1)

In [80]:
svrp2_k_frame, svrp2_y_frame, models = sep.stratified_cat_and_cont_categories_shuffle_split('support vector reg poly2', 'unharm_mix_svrp2',   SVR(C=1.0, kernel='poly', degree =2, epsilon=0.2), ml_matrix, X[:,1:], y)
svrp2_k_frame

/scratch/radv/mdijsselhof/cvasl/extended_harm_paper/../cvasl/seperated.py:1055: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  mod_results = pd.concat([mod_results, mod_results_current_fold])
/scratch/radv/mdijsselhof/cvasl/extended_harm_paper/../cvasl/seperated.py:1055: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  mod_results = pd.concat([mod_results, mod_results_current_fold])
/scratch/radv/mdijsselhof/cvasl/extended_harm_paper/../cvasl/seperated.py:1055: FutureWarning: The behavior of DataFra

,algorithm,fold,file_name,mae,r2,explained_variance
0,support vector reg poly2-0,0,unharm_mix_svrp2.0,11.239196,0.368743,0.377737
0,support vector reg poly2-1,1,unharm_mix_svrp2.1,11.850575,0.262237,0.266797
0,support vector reg poly2-2,2,unharm_mix_svrp2.2,11.146394,0.339794,0.350553
0,support vector reg poly2-3,3,unharm_mix_svrp2.3,11.651115,0.327104,0.330351
0,support vector reg poly2-4,4,unharm_mix_svrp2.4,11.476728,0.330554,0.335044


In [81]:
avg_svrp2 = sep.avg_k_folds(svrp2_k_frame)
avg_svrp2

,algorithm,file_name,mae,r2,explained_variance
0,0 support vector reg poly2-0 0 support v...,0 unharm_mix_svrp2.0 0 unharm_mix_svrp2....,11.472802,0.325686,0.332096


In [82]:
svrp2_y_frame

,y_test,y_pred
0,27.550000,44.211645
1,52.495890,51.253110
2,31.372603,42.388578
3,50.345205,53.267131
4,77.593735,56.290725
...,...,...
272,69.424658,63.011020
273,48.730000,47.944453
274,60.183060,47.928323
275,40.355191,53.247318


In [83]:
svrp2 = models[0]
svrp2[0]

SVR(degree=2, epsilon=0.2, kernel='poly')

In [84]:
eregr_k_frame, eregr_y_frame, models = sep.stratified_cat_and_cont_categories_shuffle_split('elasticnetCV', 'unharm_mix_eregr',  ElasticNetCV(cv=5, random_state=12), ml_matrix, X[:,1:], y)
eregr_k_frame

/scratch/radv/mdijsselhof/cvasl/extended_harm_paper/../cvasl/seperated.py:1055: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  mod_results = pd.concat([mod_results, mod_results_current_fold])
/scratch/radv/mdijsselhof/cvasl/extended_harm_paper/../cvasl/seperated.py:1055: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  mod_results = pd.concat([mod_results, mod_results_current_fold])
/scratch/radv/mdijsselhof/cvasl/extended_harm_paper/../cvasl/seperated.py:1055: FutureWarning: The behavior of DataFra

,algorithm,fold,file_name,mae,r2,explained_variance
0,elasticnetCV-0,0,unharm_mix_eregr.0,10.441085,0.446807,0.448028
0,elasticnetCV-1,1,unharm_mix_eregr.1,11.292337,0.338219,0.338220
0,elasticnetCV-2,2,unharm_mix_eregr.2,10.344996,0.427410,0.430534
0,elasticnetCV-3,3,unharm_mix_eregr.3,10.940735,0.402152,0.402158
0,elasticnetCV-4,4,unharm_mix_eregr.4,10.739511,0.403711,0.403712


In [85]:
eregr_k_frame.to_csv(output_folder + '/unharmonized_eregr_k_frame.csv')

In [86]:
avg_eregr = sep.avg_k_folds(eregr_k_frame)
avg_eregr

,algorithm,file_name,mae,r2,explained_variance
0,0 elasticnetCV-0 0 elasticnetCV-1 0 e...,0 unharm_mix_eregr.0 0 unharm_mix_eregr....,10.751733,0.40366,0.40453


In [87]:
eregr_y_frame

,y_test,y_pred
0,27.550000,43.576809
1,52.495890,51.804997
2,31.372603,42.406252
3,50.345205,55.066015
4,77.593735,59.102229
...,...,...
272,69.424658,67.353476
273,48.730000,48.616333
274,60.183060,48.441480
275,40.355191,54.531479


In [88]:
eregr = models[0]
eregr[0]

ElasticNetCV(cv=5, random_state=12)

In [89]:
etreg_k_frame, etreg_y_frame, models = sep.stratified_cat_and_cont_categories_shuffle_split('extra trees', 'unharm_mix_etreg',  ExtraTreesRegressor(n_estimators=100, random_state=0), ml_matrix, X[:,1:], y)
etreg_k_frame

/scratch/radv/mdijsselhof/cvasl/extended_harm_paper/../cvasl/seperated.py:1055: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  mod_results = pd.concat([mod_results, mod_results_current_fold])
/scratch/radv/mdijsselhof/cvasl/extended_harm_paper/../cvasl/seperated.py:1055: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  mod_results = pd.concat([mod_results, mod_results_current_fold])
/scratch/radv/mdijsselhof/cvasl/extended_harm_paper/../cvasl/seperated.py:1055: FutureWarning: The behavior of DataFra

,algorithm,fold,file_name,mae,r2,explained_variance
0,extra trees-0,0,unharm_mix_etreg.0,5.162967,0.855429,0.855677
0,extra trees-1,1,unharm_mix_etreg.1,4.851738,0.870823,0.870916
0,extra trees-2,2,unharm_mix_etreg.2,5.074833,0.854355,0.854852
0,extra trees-3,3,unharm_mix_etreg.3,5.202731,0.850795,0.850822
0,extra trees-4,4,unharm_mix_etreg.4,5.244762,0.834009,0.834029


In [90]:
etreg_k_frame.to_csv(output_folder + '/unharomized_etreg_k_frame.csv')

In [91]:
avg_etreg = sep.avg_k_folds(etreg_k_frame)
avg_etreg

,algorithm,file_name,mae,r2,explained_variance
0,0 extra trees-0 0 extra trees-1 0 ext...,0 unharm_mix_etreg.0 0 unharm_mix_etreg....,5.107406,0.853082,0.853259


In [92]:
etreg_y_frame

,y_test,y_pred
0,27.550000,31.526055
1,52.495890,50.411040
2,31.372603,41.381270
3,50.345205,54.790364
4,77.593735,67.716469
...,...,...
272,69.424658,71.406362
273,48.730000,40.437157
274,60.183060,56.502528
275,40.355191,35.930549


In [93]:
etreg_y_frame.to_csv(output_folder + '/unharmonized_etreg_y_frame.csv')

In [94]:
etreg = models[0]
etreg[0]

ExtraTreesRegressor(random_state=0)

In [95]:
## optional save models
#joblib.dump(etreg[0], ('../result_models/'+ 'unharm_mix_etreg0.sav'))
#joblib.dump(etreg[1], ('../result_models/'+ 'unharm_mix_etreg1.sav'))
#joblib.dump(etreg[2], ('../result_models/'+ 'unharm_mix_etreg2.sav'))
#joblib.dump(etreg[3], ('../result_models/'+ 'unharm_mix_etreg3.sav'))
#joblib.dump(etreg[4], ('../result_models/'+ 'unharm_mix_etreg4.sav'))

Show results ON AVERAGE for each model

In [96]:
mixed_based_unharmonized_on_testmix =pd.concat([avg_linr,
                   avg_llreg,
                   avg_dtree,
                   avg_regr,
                   avg_svrp2,
                   avg_eregr,
                   avg_etreg],
                  axis=0)
mixed_based_unharmonized_on_testmix

,algorithm,file_name,mae,r2,explained_variance
0,0 linear regression-0 0 linear regressio...,0 unharm_mix_linr.0 0 unharm_mix_linr.1 ...,5.562186,0.827286,0.827902
0,0 lasso regression-0 0 lasso regression-...,0 unharm_mix_llreg.0 0 unharm_mix_llreg....,5.561026,0.827710,0.828357
0,0 decision tree-0 0 decision tree-1 0 ...,0 unharm_mix_dtree.0 0 unharm_mix_dtree....,7.282041,0.683594,0.685599
0,0 MLP regression-0 0 MLP regression-1 0 ...,0 unharm_mix_regr.0 0 unharm_mix_regr.1 ...,7.447758,0.692654,0.710466
0,0 support vector reg poly2-0 0 support v...,0 unharm_mix_svrp2.0 0 unharm_mix_svrp2....,11.472802,0.325686,0.332096
0,0 elasticnetCV-0 0 elasticnetCV-1 0 e...,0 unharm_mix_eregr.0 0 unharm_mix_eregr....,10.751733,0.403660,0.404530
0,0 extra trees-0 0 extra trees-1 0 ext...,0 unharm_mix_etreg.0 0 unharm_mix_etreg....,5.107406,0.853082,0.853259


In [97]:
mixed_based_unharmonized_on_testmix.to_csv(output_folder + '/mixed_based_unharmonized_on_testmix_AVERAGES.csv')

## Run mixed models on other datasets (All TOP- StrokeMRI blended model on INsight and Sabre)
now we will make some models based on all top an strokeMRI data together

In [98]:
TOPMRI = pd.concat([TOP, StrokeMRI])
TOPMRI.head(3)

,participant_id,age,sex,gm_vol,wm_vol,csf_vol,gm_icvratio,gmwm_icvratio,wmhvol_wmvol,wmh_count,deepwm_b_cov,aca_b_cov,mca_b_cov,pca_b_cov,totalgm_b_cov,deepwm_b_cbf,aca_b_cbf,mca_b_cbf,pca_b_cbf,totalgm_b_cbf
0,sub-0668_1,50.40,1,0.64252,0.48470,0.26711,0.46081,0.80843,0.004506,18,0.26694,0.17747,0.20996,0.24951,0.24191,24.9902,90.5294,80.9975,57.9326,72.1759
1,sub-0532_1,37.02,1,0.71890,0.59546,0.31236,0.44193,0.80798,0.002501,24,0.25342,0.17620,0.23222,0.21187,0.23518,24.2061,101.2955,90.3336,67.8547,80.1958
2,sub-0529_1,30.57,1,0.66610,0.45798,0.28326,0.47330,0.79873,0.005144,20,0.27421,0.19233,0.20855,0.19810,0.21863,26.5891,108.6481,99.6353,72.4565,87.2374


In [99]:
ml_matrix = TOPMRI.drop('participant_id', axis=1)
X = ml_matrix.drop('age', axis =1)
X = X.values
X = X.astype('float')
y = ml_matrix['age'].values
y=y.astype('float')


In [100]:
# now we decide to do no test train split, rather take all
X_train = X
y_train = y

In [101]:
MIXlinr = LinearRegression()
MIXlinr.fit(X_train, y_train)

LinearRegression()

In [102]:
MIXllreg = linear_model.LassoLars(alpha=0.01)
MIXllreg.fit(X_train, y_train)

LassoLars(alpha=0.01)

In [103]:
MIXeregr = ElasticNetCV(cv=5, random_state=17)
MIXeregr.fit(X_train, y_train)


ElasticNetCV(cv=5, random_state=17)

In [104]:
MIXetreg = ExtraTreesRegressor(n_estimators=100, random_state=0)
MIXetreg.fit(X_train, y_train)

ExtraTreesRegressor(random_state=0)

In [105]:
##  Save these four best models

In [106]:
## optional save models
#joblib.dump(MIXlinr, ('../result_models/'+ 'unharm_mix_MIXlinr.sav'))
#joblib.dump(MIXllreg, ('../result_models/'+ 'unharm_mix_MIXllreg1.sav'))
#joblib.dump(MIXeregr, ('../result_models/'+ 'unharm_mix_MIXeregr3.sav'))
#joblib.dump(MIXetreg, ('../result_models/'+ 'unharm_mix_MIXetreg4.sav'))

# Running mixed model over SABRE dataset

In [107]:
sabre_ml_matrix = SABRE.drop('participant_id', axis=1)
X_sabre = sabre_ml_matrix.drop('age', axis =1)
X_sabre = X_sabre.values
X_sabre = X_sabre.astype('float')
y_sabre = sabre_ml_matrix['age'].values
y_sabre=y_sabre.astype('float')

In [108]:
X_sabre_test = X_sabre
y_sabre_test = y_sabre

In [109]:
y_sabre_pred = MIXlinr.predict(X_sabre_test)

In [110]:
data= [[
    'Linear Reg',
    'unharm_mix_MIXlinr.sav',
    mean_absolute_error(y_sabre_test, y_sabre_pred),
    MIXlinr.score(X_sabre_test,y_sabre_test),
    metrics.explained_variance_score(y_sabre_test, y_sabre_pred)]]
linr_results_sabre = pd.DataFrame(data, columns=['algorithm','file_name','mae', 'r2', 'explained_variance'])
linr_results_sabre

,algorithm,file_name,mae,r2,explained_variance
0,Linear Reg,unharm_mix_MIXlinr.sav,35.213167,-46.555899,-19.455256


In [111]:
data = {'real': y_sabre_test, 'predicted': y_sabre_pred}
y_frame_linr_sabre = pd.DataFrame(data)
y_frame_linr_sabre

,real,predicted
0,79.0,29.911459
1,70.0,47.936747
2,62.0,44.364579
3,78.0,47.404238
4,68.0,-39.838887
...,...,...
701,66.0,35.608471
702,72.0,32.984624
703,73.0,27.275481
704,65.0,40.375821


In [112]:
y_frame_linr_sabre.to_csv(output_folder + '/unharm_y_frame_linr_sabre_MIX.csv')

In [113]:
y_sabre_pred = MIXllreg.predict(X_sabre_test)

In [114]:
data= [[
    'Lasso',
    'unharm_mix_MIXlassor.sav',
    mean_absolute_error(y_sabre_test, y_sabre_pred),
    MIXllreg.score(X_sabre_test,y_sabre_test),
    metrics.explained_variance_score(y_sabre_test, y_sabre_pred)]]
llreg_results_sabre = pd.DataFrame(data, columns=['algorithm','file_name','mae', 'r2', 'explained_variance'])
llreg_results_sabre

,algorithm,file_name,mae,r2,explained_variance
0,Lasso,unharm_mix_MIXlassor.sav,25.69999,-21.593511,-6.875801


In [115]:
data = {'real': y_sabre_test, 'predicted': y_sabre_pred}
y_frame_llreg_sabre = pd.DataFrame(data)
y_frame_llreg_sabre

,real,predicted
0,79.0,39.336573
1,70.0,53.826394
2,62.0,52.271255
3,78.0,51.201873
4,68.0,-6.480717
...,...,...
701,66.0,43.158678
702,72.0,40.885251
703,73.0,38.254702
704,65.0,46.945776


In [116]:
y_frame_llreg_sabre.to_csv(output_folder + '/unharm_y_frame_llreg_sabre_MIX.csv')

In [117]:
data= [[
    'ElasticnetCV',
    'unharm_mri_elasticregr.sav',
    mean_absolute_error(y_sabre_test, y_sabre_pred),
    MIXeregr.score(X_sabre_test,y_sabre_test),
    metrics.explained_variance_score(y_sabre_test, y_sabre_pred)]]
eregr_results_sabre = pd.DataFrame(data, columns=['algorithm','file_name','mae', 'r2', 'explained_variance'])
eregr_results_sabre

,algorithm,file_name,mae,r2,explained_variance
0,ElasticnetCV,unharm_mri_elasticregr.sav,25.69999,-6.00718,-6.875801


In [118]:
y_sabre_pred = MIXetreg.predict(X_sabre_test)

In [119]:
data= [[
    'Extra trees',
    'unharm_mri_etreg.sav',
    mean_absolute_error(y_sabre_test, y_sabre_pred),
    MIXetreg.score(X_sabre_test,y_sabre_test),
    metrics.explained_variance_score(y_sabre_test, y_sabre_pred)]]
etreg_results_sabre = pd.DataFrame(data, columns=['algorithm','file_name','mae', 'r2', 'explained_variance'])
etreg_results_sabre

,algorithm,file_name,mae,r2,explained_variance
0,Extra trees,unharm_mri_etreg.sav,7.72702,-1.095742,-0.056009


In [120]:
data = {'real': y_sabre_test, 'predicted': y_sabre_pred}
y_frame_etregr_sabre = pd.DataFrame(data)
y_frame_etregr_sabre

,real,predicted
0,79.0,70.797948
1,70.0,71.174930
2,62.0,68.155861
3,78.0,64.217257
4,68.0,61.115701
...,...,...
701,66.0,50.342667
702,72.0,59.163687
703,73.0,68.067074
704,65.0,58.660931


In [121]:
y_frame_etregr_sabre.to_csv(output_folder + '/y_frame_etregr_sabre_MIX.csv')

In [122]:
mix_based_unharmonized_on_sabre =pd.concat([linr_results_sabre,
                   llreg_results_sabre,
                   eregr_results_sabre,
                  etreg_results_sabre],
                  axis=0)
mix_based_unharmonized_on_sabre

,algorithm,file_name,mae,r2,explained_variance
0,Linear Reg,unharm_mix_MIXlinr.sav,35.213167,-46.555899,-19.455256
0,Lasso,unharm_mix_MIXlassor.sav,25.699990,-21.593511,-6.875801
0,ElasticnetCV,unharm_mri_elasticregr.sav,25.699990,-6.007180,-6.875801
0,Extra trees,unharm_mri_etreg.sav,7.727020,-1.095742,-0.056009


In [123]:
mix_based_unharmonized_on_sabre.to_csv(output_folder + '/mix_based_unharmonized_on_sabre.csv')

# Running mixed model over Insight46 dataset

In [124]:
insight_ml_matrix = Insight46.drop('participant_id', axis=1)
X_insight = insight_ml_matrix.drop('age', axis =1)
X_insight = X_insight.values
X_insight = X_insight.astype('float')
y_insight = insight_ml_matrix['age'].values
y_insight= y_insight.astype('float')

In [125]:
X_insight_test = X_insight
y_insight_test = y_insight

In [126]:
y_insight_pred = MIXlinr.predict(X_insight_test)

In [127]:
data= [[
    'Linear Reg',
    'unharm_mix_linr0.sav',
    mean_absolute_error(y_insight_test, y_insight_pred),
    MIXlinr.score(X_insight_test,y_insight_test),
    metrics.explained_variance_score(y_insight_test, y_insight_pred)]]
linr_results_insight = pd.DataFrame(data, columns=['algorithm','file_name','mae', 'r2', 'explained_variance'])
linr_results_insight

,algorithm,file_name,mae,r2,explained_variance
0,Linear Reg,unharm_mix_linr0.sav,34.928583,-14176.29315,-13417.636832


In [128]:
data = {'real': y_insight_test, 'predicted': y_insight_pred}
y_frame_linr_insight = pd.DataFrame(data)
y_frame_linr_insight

,real,predicted
0,71.072222,53.438880
1,73.000000,37.873062
2,74.000000,48.306209
3,71.644444,50.427440
4,71.694444,53.128711
...,...,...
612,72.000000,65.319793
613,73.000000,-262.457223
614,72.000000,70.115782
615,70.605556,51.376688


In [129]:
y_frame_linr_insight.to_csv(output_folder + '/y_frame_linr_insight_MIX.csv')

In [130]:
y_insight_pred = MIXllreg.predict(X_insight_test)

In [131]:
data= [[
    'Lasso',
    'unharm_mix_lassor0.sav',
    mean_absolute_error(y_insight_test, y_insight_pred),
    MIXllreg.score(X_insight_test,y_insight_test),
    metrics.explained_variance_score(y_insight_test, y_insight_pred)]]
llreg_results_insight = pd.DataFrame(data, columns=['algorithm','file_name','mae', 'r2', 'explained_variance'])
llreg_results_insight

,algorithm,file_name,mae,r2,explained_variance
0,Lasso,unharm_mix_lassor0.sav,25.94615,-4652.555289,-4234.223629


In [132]:
data = {'real': y_insight_test, 'predicted': y_insight_pred}
y_frame_llreg_insight = pd.DataFrame(data)
y_frame_llreg_insight

,real,predicted
0,71.072222,58.480226
1,73.000000,48.207068
2,74.000000,52.925082
3,71.644444,55.276125
4,71.694444,54.187854
...,...,...
612,72.000000,67.887244
613,73.000000,-167.612043
614,72.000000,68.907094
615,70.605556,54.098795


In [133]:
y_frame_llreg_insight.to_csv(output_folder + '/y_frame_llreg_insight_MIX.csv')

In [134]:
y_insight_pred = MIXeregr.predict(X_insight_test)

In [135]:
data= [[
    'ElasticnetCV',
    'unharm_mri_elasticregr.sav',
    mean_absolute_error(y_insight_test, y_insight_pred),
    MIXeregr.score(X_insight_test,y_insight_test),
    metrics.explained_variance_score(y_insight_test, y_insight_pred)]]
eregr_results_insight = pd.DataFrame(data, columns=['algorithm','file_name','mae', 'r2', 'explained_variance'])
eregr_results_insight

,algorithm,file_name,mae,r2,explained_variance
0,ElasticnetCV,unharm_mri_elasticregr.sav,20.839383,-349.632843,-54.892047


In [136]:
data = {'real': y_insight_test, 'predicted': y_insight_pred}
y_frame_eregr_insight = pd.DataFrame(data)
y_frame_eregr_insight

,real,predicted
0,71.072222,48.053387
1,73.000000,54.878212
2,74.000000,45.456221
3,71.644444,48.745270
4,71.694444,44.242711
...,...,...
612,72.000000,66.015592
613,73.000000,74.204722
614,72.000000,63.949207
615,70.605556,43.213231


In [137]:
y_frame_eregr_insight.to_csv(output_folder + '/y_frame_eregr_insight_MIX.csv')

In [138]:
y_insight_pred = MIXetreg.predict(X_insight_test)

In [139]:
data= [[
    'Extra trees',
    'unharm_mri_etreg.sav',
    mean_absolute_error(y_insight_test, y_insight_pred),
    MIXetreg.score(X_insight_test,y_insight_test),
    metrics.explained_variance_score(y_insight_test, y_insight_pred)]]
etreg_results_insight = pd.DataFrame(data, columns=['algorithm','file_name','mae', 'r2', 'explained_variance'])
etreg_results_insight

,algorithm,file_name,mae,r2,explained_variance
0,Extra trees,unharm_mri_etreg.sav,10.13042,-92.083366,-24.521995


In [140]:
data = {'real': y_insight_test, 'predicted': y_insight_pred}
y_frame_etregr_insight = pd.DataFrame(data)
y_frame_etregr_insight

,real,predicted
0,71.072222,63.635311
1,73.000000,68.019945
2,74.000000,65.898996
3,71.644444,61.446249
4,71.694444,60.065436
...,...,...
612,72.000000,67.905392
613,73.000000,67.089732
614,72.000000,74.876648
615,70.605556,54.888766


In [141]:
y_frame_etregr_insight.to_csv(output_folder + '/y_frame_etregr_insight_MIX.csv')

In [142]:
mix_based_unharmonized_on_insight =pd.concat([linr_results_insight,
                   llreg_results_insight,
                   eregr_results_insight,
                   etreg_results_insight],
                  axis=0)
mix_based_unharmonized_on_insight

,algorithm,file_name,mae,r2,explained_variance
0,Linear Reg,unharm_mix_linr0.sav,34.928583,-14176.293150,-13417.636832
0,Lasso,unharm_mix_lassor0.sav,25.946150,-4652.555289,-4234.223629
0,ElasticnetCV,unharm_mri_elasticregr.sav,20.839383,-349.632843,-54.892047
0,Extra trees,unharm_mri_etreg.sav,10.130420,-92.083366,-24.521995


In [143]:
mix_based_unharmonized_on_insight.to_csv(output_folder + '/mix_based_unharmonized_on_insight.csv')

# Run mixed model on EDIS

In [144]:
edis_ml_matrix = EDIS.drop('participant_id', axis=1)
X_edis = edis_ml_matrix.drop('age', axis =1)
X_edis = X_edis.values
X_edis = X_edis.astype('float')
y_edis = edis_ml_matrix['age'].values
y_edis=y_edis.astype('float')

In [145]:
X_edis_test = X_edis
y_edis_test = y_edis

In [146]:
y_edis_pred = MIXlinr.predict(X_edis_test)

In [147]:
data= [[
    'Linear Reg',
    'unharm_mix_MIXlinr.sav',
    mean_absolute_error(y_edis_test, y_edis_pred),
    MIXlinr.score(X_edis_test,y_edis_test),
    metrics.explained_variance_score(y_edis_test, y_edis_pred)]]
linr_results_edis = pd.DataFrame(data, columns=['algorithm','file_name','mae', 'r2', 'explained_variance'])
linr_results_edis

,algorithm,file_name,mae,r2,explained_variance
0,Linear Reg,unharm_mix_MIXlinr.sav,97.230107,-2313.274483,-2115.85209


In [148]:
data = {'real': y_edis_test, 'predicted': y_edis_pred}
y_frame_linr_edis = pd.DataFrame(data)
y_frame_linr_edis

,real,predicted
0,66.0,-68.198516
1,71.0,-16.723679
2,84.0,-47.343901
3,69.0,0.129149
4,69.0,29.204752
...,...,...
341,68.0,35.686531
342,62.0,7.470796
343,68.0,20.099561
344,71.0,16.578332


In [149]:
y_frame_linr_edis.to_csv(output_folder + '/unharm_y_frame_linr_edis_MIX.csv')

In [150]:
y_edis_pred = MIXllreg.predict(X_edis_test)

In [151]:
data= [[
    'Lasso',
    'unharm_mix_MIXlassor.sav',
    mean_absolute_error(y_edis_test, y_edis_pred),
    MIXllreg.score(X_edis_test,y_edis_test),
    metrics.explained_variance_score(y_edis_test, y_edis_pred)]]
llreg_results_edis = pd.DataFrame(data, columns=['algorithm','file_name','mae', 'r2', 'explained_variance'])
llreg_results_edis

,algorithm,file_name,mae,r2,explained_variance
0,Lasso,unharm_mix_MIXlassor.sav,72.942803,-1057.555363,-942.151955


In [152]:
data = {'real': y_edis_test, 'predicted': y_edis_pred}
y_frame_llreg_edis = pd.DataFrame(data)
y_frame_llreg_edis

,real,predicted
0,66.0,-44.486653
1,71.0,-13.816355
2,84.0,-20.620842
3,69.0,15.782148
4,69.0,41.284328
...,...,...
341,68.0,38.414300
342,62.0,14.520096
343,68.0,32.177672
344,71.0,24.423610


In [153]:
y_frame_llreg_edis.to_csv(output_folder + '/unharm_y_frame_llreg_edis_MIX.csv')

In [154]:
data= [[
    'ElasticnetCV',
    'unharm_mri_elasticregr.sav',
    mean_absolute_error(y_edis_test, y_edis_pred),
    MIXeregr.score(X_edis_test,y_edis_test),
    metrics.explained_variance_score(y_edis_test, y_edis_pred)]]
eregr_results_edis = pd.DataFrame(data, columns=['algorithm','file_name','mae', 'r2', 'explained_variance'])
eregr_results_edis

,algorithm,file_name,mae,r2,explained_variance
0,ElasticnetCV,unharm_mri_elasticregr.sav,72.942803,-27.493098,-942.151955


In [155]:
y_edis_pred = MIXetreg.predict(X_edis_test)

In [156]:
data= [[
    'Extra trees',
    'unharm_mri_etreg.sav',
    mean_absolute_error(y_edis_test, y_edis_pred),
    MIXetreg.score(X_edis_test,y_edis_test),
    metrics.explained_variance_score(y_edis_test, y_edis_pred)]]
etreg_results_edis = pd.DataFrame(data, columns=['algorithm','file_name','mae', 'r2', 'explained_variance'])
etreg_results_edis

,algorithm,file_name,mae,r2,explained_variance
0,Extra trees,unharm_mri_etreg.sav,7.478491,-1.467841,-0.671659


In [157]:
data = {'real': y_edis_test, 'predicted': y_edis_pred}
y_frame_etregr_edis = pd.DataFrame(data)
y_frame_etregr_edis

,real,predicted
0,66.0,45.617302
1,71.0,51.781022
2,84.0,66.191718
3,69.0,64.299769
4,69.0,67.060219
...,...,...
341,68.0,65.115909
342,62.0,51.788395
343,68.0,65.499662
344,71.0,68.737795


In [158]:
y_frame_etregr_edis.to_csv(output_folder + '/y_frame_etregr_edis_MIX.csv')

In [159]:
mix_based_unharmonized_on_edis =pd.concat([linr_results_edis,
                   llreg_results_edis,
                   eregr_results_edis,
                  etreg_results_edis],
                  axis=0)
mix_based_unharmonized_on_edis

,algorithm,file_name,mae,r2,explained_variance
0,Linear Reg,unharm_mix_MIXlinr.sav,97.230107,-2313.274483,-2115.852090
0,Lasso,unharm_mix_MIXlassor.sav,72.942803,-1057.555363,-942.151955
0,ElasticnetCV,unharm_mri_elasticregr.sav,72.942803,-27.493098,-942.151955
0,Extra trees,unharm_mri_etreg.sav,7.478491,-1.467841,-0.671659


In [160]:
mix_based_unharmonized_on_edis.to_csv(output_folder + '/mix_based_unharmonized_on_edis.csv')